# **NLP From Scratch**: Translation with a Sequence to Sequence Network and Attention
  + In this project we will be teaching a neural network to translate from French to English.

    -  [KEY: > input, = target, < output]
      
       ####  > il est en train de peindre un tableau .  
       ####  = he is painting a picture .
       ####  < he is painting a picture . 

In [1]:
from __future__ import  unicode_literals, print_function, division

from io import open
import unicodedata
import re
import random 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Edit with Google Colab,, put images and explanations

In [2]:
S = 0 #start of sentence
E = 1 #end of sentnce

class Lang:
    def __init__(self, langName):
        self.langName = langName
        self.wtoi = {} #word to indx
        self.wordToCount = {}
        self.itow = {0:'<S>', 1:'<E'} # indx to word
        self.nwords = 2
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.wtoi:
            self.wtoi[word] = self.nwords
            self.wordToCount[word] = 1
            self.itow[self.nwords] = word
            self.nwords += 1  # Count S and E
        
        else:
            self.wordToCount[word] += 1
    




# The files are all in Unicode, to simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizStr(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r'([.!?])', r" \1", s)
    s = re.sub(r'[^a-zA-Z!?]+', r' ', s)
    return s.strip()


#### To read the data file we will split the file into lines, and then split lines into pairs. The files are all English → Other Language, so if we want to translate from Other Language → English I added the reverse flag to reverse the pairs.

In [4]:

def loadData(filePath, reverse=False):
    pairs = []
    with open(filePath, 'r', encoding='utf-8') as f:
        for line in f:
            eng, fre = line.strip().split('\t') #english french

            if reverse:
                pairs.append([fre, eng])
            
            else:
                pairs.append([eng, fre])

    return pairs    

##### Since there are a lot of example sentences and we want to train something quickly, we’ll trim the data set to only relatively short and simple sentences. Here the maximum length is 10 words (that includes ending punctuation) and we’re filtering to sentences that translate to the form “I am” or “He is” etc. (accounting for apostrophes replaced earlier).

In [5]:
MAX_LEN = 10

engPrefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filtrPair(p):
    return len(p[0].split(' ')) < MAX_LEN and len(p[1].split(' ')) < MAX_LEN and p[1].startswith(engPrefixes)

def filtrPairs(pairs):
    return [pair for pair in pairs if filtrPair(pair)]


# The full process for preparing the data is:
   
   + Read text file and split into lines, split lines into pairs

   + Normalize text, filter by length and content

   + Make word lists from sentences in pairs

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizStr(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))  # Check if pairs were loaded correctly

    pairs = filtrPairs(pairs)
    print("Trimmed to %s sentence pairs after filtering" % len(pairs))  # Check after filtering
    print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.langName, input_lang.nwords)
    print(output_lang.langName, output_lang.nwords)
    
    if not pairs:
        print("Warning: No pairs found after filtering. Check data or filtering criteria.")
    
    # The rest of your code for counting words etc.
    return input_lang, output_lang, pairs


In [8]:
inputLang, outputLang, pairs = prepareData('eng', 'fra', reverse=True)
print(random.choice(pairs))


Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs after filtering
Counting words...
Counted words:
fra 4601
eng 2991
['je pense a mes vacances', 'i am thinking of my vacation']


# The Encoder
   + The encoder of a seq2seq network is a RNN that outputs some value for every word from the input sentence. 
   
   + For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, nInput, nHidden, dropoutP=0.1):
        super(EncoderRNN, self).__init__()
        self.nHidden = nHidden

        self.emb = nn.Embedding(nInput, nHidden)
        self.gru = nn.GRU(nHidden, nHidden, batch_first=True)
        self.dropout = nn.Dropout(dropoutP)

    
    def forward(self, x):
        emb = self.dropout(self.emb(x))
        x, hidden = self.gru(emb)
        return x, hidden
    

# The Decoder
   + The decoder is another RNN that takes the encoder output vector(s) and outputs a sequence of words to create the translation.

## Simple Decoder
   + In the simplest seq2seq decoder we use only last output of the encoder. 
    
   + This last output is sometimes called the context vector as it encodes context from the entire sequence. 
   
   + This context vector is used as the initial hidden state of the decoder.

  * At every step of decoding, the decoder is given an input token and hidden state. 
  
  * The initial input token is the start-of-string <SOS> token, and the first hidden state is the context vector (the encoder’s last hidden state).

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, nHidden, nOutput):
        super(DecoderRNN, self).__init__()
        self.emb = nn.Embedding(nOutput, nHidden)
        self.gru = nn.GRU(nHidden, nHidden, batch_first=True)
        self.out = nn.Linear(nHidden, nOutput)
    
    def forward(self, encOutput, encHidden, targtTensor=None):
        batchSiz = encOutput.size(0)
        decodInput = torch.empty(batchSiz, 1, dtype=torch.long, device=device).fill_(S)
        
        decodHidden = encHidden
        decodOutputs = []

        for i in range(MAX_LEN):
            decodOutput, decodHidden = self.forwardStep(decodInput, decodHidden)
            decodOutputs.append(decodOutput)

            if targtTensor is not None:
                decodInput = targtTensor[:, i].unsqueeze(1)
            
            else:
                _, topi = decodOutput.topk(i)
                decodInput = topi.squeeze(-1).detach()
        
        decodOutputs = torch.cat(decodOutputs, dim=1)
        decodOutputs = F.log_softmax(decodOutputs, dim=1)
        
        return decodOutputs, decodHidden, None 

    def forwardStep(self, x, hidden):
        x = self.emb(x)
        x = F.relu(x)
        x, hidden = self.gru(x, hidden)
        x = self.out(x)
        return x, hidden
    



# Training
   
## Preparing Training Data
        
   + To train, for each pair we will need an input tensor (indexes of the words in the input sentence) and target tensor (indexes of the words in the target sentence). 
    
   + While creating these vectors we will append the EOS token to both sequences.

In [11]:
def indexFromSentence(lang, sentenc):
    return [lang.wtoi[word] for word in sentenc.split(' ')]

def tensorToSentence(lang, sentence):
    ix = indexFromSentence(lang, sentence)
    ix.append(E)

    return torch.tensor(ix, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    inputTensor = tensorsFromPair(inputLang, pair[0])
    targtTensor = tensorsFromPair(outputLang, pair[1])

    return (inputTensor, targtTensor)

def getDataLoader(batchSiz):
    inputLang, outputLang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    inputIds = np.zeros((n, MAX_LEN), dtype=np.int32)
    tatrgtIds = np.zeros((n, MAX_LEN), dtype=np.int32)

    for ix, (inp, targ) in enumerate(pairs):
        inpIds = indexFromSentence(inputLang, inp)
        targIds = indexFromSentence(outputLang, targ)
        inpIds.append(E)
        targIds.append(E)
        inputIds[ix, :len(inpIds)] = inpIds
        tatrgtIds[ix, :len(targIds)] = targIds 
    
    trainData = TensorDataset(torch.LongTensor(inputIds).to(device), torch.LongTensor(tatrgtIds).to(device))

    trainSample = RandomSampler(trainData)
    trainDataLoader = DataLoader(trainData, sampler=trainSample, batch_size=batchSiz)

    return inputLang, outputLang, trainDataLoader




# Training the Model
   
   + To train we run the input sentence through the encoder, and keep track of every output and the latest hidden state. 
   
   + Then the decoder is given the <SOS> token as its first input, and the last hidden state of the encoder as its first hidden state.

   + “Teacher forcing” is the concept of using the real target outputs as each next input, instead of using the decoder’s guess as the next input. 
   
   + Using teacher forcing causes it to converge faster but when the trained network is exploited, it may exhibit instability.

* You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - intuitively it has learned to represent the output grammar and can “pick up” the meaning once the teacher tells it the first few words, but it has not properly learned how to create the sentence from the translation in the first place.

* Because of the freedom PyTorch’s autograd gives us, we can randomly choose to use teacher forcing or not with a simple if statement. Turn teacher_forcing_ratio up to use more of it.

In [12]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

+ This is a helper function to print time elapsed and estimated time remaining given the current time and progress %.

In [13]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## The whole training process looks like this:

  + Start a timer

  + Initialize optimizers and criterion

  + Create set of training pairs

  + Start empty losses array for plotting

* Then we call train many times and occasionally print the progress (% of examples, time so far, estimated time) and average loss.

In [21]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# Plotting results

  + Plotting is done with matplotlib, using the array of loss values plot_losses saved while training.

In [22]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Evaluation
   
   + Evaluation is mostly the same as training, but there are no targets so we simply feed the decoder’s predictions back to itself for each step. 
   
   + Every time it predicts a word we add it to the output string, and if it predicts the EOS token we stop there. 
   
   + We also store the decoder’s attention outputs for display later.



In [23]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorToSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == E:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.itow[idx.item()])
    return decoded_words, decoder_attn

+ We can evaluate random sentences from the training set and print out the input, target, and output to make some subjective quality judgements:

In [25]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], inputLang, outputLang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# Training and Evaluating
  
  + With all these helper functions in place (it looks like extra work, but it makes it easier to run multiple experiments) we can actually initialize a network and start training.

  + Remember that the input sentences were heavily filtered. 
  
  + For this small dataset we can use relatively small networks of 256 hidden nodes and a single GRU layer. 
  
  + After about 40 minutes on a MacBook CPU we’ll get some reasonable results.

In [24]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = getDataLoader(batch_size)

encoder = EncoderRNN(input_lang.nwords, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.nwords).to(device)

train(train_dataloader, encoder, decoder, 10, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs after filtering
Counting words...
Counted words:
fra 4601
eng 2991
1m 31s (- 1m 31s) (5 50%) 0.7135
3m 3s (- 0m 0s) (10 100%) 0.5591


### + Set dropout layers to eval mode

In [26]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> nous nous dirigeons dans la bonne direction
= we re headed in the right direction


ValueError: GRU: Expected input to be 2D or 3D, got 4D instead